### Define the network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# defining a network
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5*5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # an affine operatio: y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:] # all dim except batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
# creating a network
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


the backward function is automatically defined using **autograd**

In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [7]:
# trying a random 32*32 input
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0879,  0.0805,  0.0186,  0.0841, -0.0566,  0.0813,  0.0127, -0.1164,
         -0.0707,  0.0291]], grad_fn=<ThAddmmBackward>)


In [8]:
# zeroing the gradients buffers of all parameters
net.zero_grad()
out.backward(torch.randn(1,10))

- torch.nn only support mini-batches
- in case of single sample, use **input.unsqueeze(0)** to fake a batch dimension

### Loss function

In [9]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.2219, grad_fn=<MseLossBackward>)


Calling loss.backward, the whole graph is differentiated w.r.t. the loss

In [10]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLu

### Backprop
All we have to do is: **loss.backward()**. You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

In [11]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0121, -0.0035,  0.0073, -0.0069, -0.0112,  0.0008])


### Update the weights

torch.optim package that implement all optimization methods

In [12]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()